## PART-2(CROPPING THE DETECTED IMAGE)

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#### 1. Importing the required libraries and reading the json file

In [2]:
import pandas as pd
import json

In [3]:
%cd 'drive/My Drive'
with open('Plates_Crop/result_vehicle_plates.json') as file:
    data = json.load(file)
dataset = pd.DataFrame(data)
dataset

/content/drive/My Drive


,frame_id,filename,objects
0,1,data/img2/0.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat..."
1,2,data/img2/1.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat..."
2,3,data/img2/10.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat..."
3,4,data/img2/100.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat..."
4,5,data/img2/101.jpg,[]
...,...,...,...
137,138,data/img2/95.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat..."
138,139,data/img2/96.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat..."
139,140,data/img2/97.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat..."
140,141,data/img2/98.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat..."


In [4]:
lis1 = []
def check(x):
    for p in x:
        if p['name']=="NumberPlate":
                    return True
    return False

dataset['plates']=dataset['objects'].transform(lambda temp:check(temp))

#### 2. Splitting the filename and storing the names of the file in a list

In [5]:
dataset = dataset[dataset['plates']]
dataset['filename'] = dataset['filename'].transform(lambda f:f.split('/')[-1])
in_book = list(dataset['filename'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
%cd 'Plates_Crop/img2'
import subprocess
proc = subprocess.Popen('ls', shell=True, stdout=subprocess.PIPE, )
output = proc.communicate()[0]
output = output.decode('utf-8').split('\n')
output = list(output)

/content/drive/My Drive/Plates_Crop/img2


In [7]:
p = set(output)-set(in_book)
for x in p:
    %rm $x

rm: missing operand
Try 'rm --help' for more information.


In [8]:
lis2 = []
def check2(x):
    for p in x:
        if p['name']=="NumberPlate":
                    return p
    return False

dataset['plate_data'] = dataset['objects'].transform(lambda temp:check2(temp))
dataset

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,frame_id,filename,objects,plates,plate_data
0,1,0.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat...",True,"{'class_id': 0, 'name': 'NumberPlate', 'relati..."
1,2,1.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat...",True,"{'class_id': 0, 'name': 'NumberPlate', 'relati..."
2,3,10.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat...",True,"{'class_id': 0, 'name': 'NumberPlate', 'relati..."
3,4,100.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat...",True,"{'class_id': 0, 'name': 'NumberPlate', 'relati..."
5,6,102.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat...",True,"{'class_id': 0, 'name': 'NumberPlate', 'relati..."
...,...,...,...,...,...
137,138,95.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat...",True,"{'class_id': 0, 'name': 'NumberPlate', 'relati..."
138,139,96.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat...",True,"{'class_id': 0, 'name': 'NumberPlate', 'relati..."
139,140,97.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat...",True,"{'class_id': 0, 'name': 'NumberPlate', 'relati..."
140,141,98.jpg,"[{'class_id': 0, 'name': 'NumberPlate', 'relat...",True,"{'class_id': 0, 'name': 'NumberPlate', 'relati..."


#### 3. Dropping the unnecessary columns

In [9]:
dataset = dataset.drop(['frame_id','objects','plates'],axis=1)
dataset['plate_data'][0]

{'class_id': 0,
 'confidence': 0.473579,
 'name': 'NumberPlate',
 'relative_coordinates': {'center_x': 0.495805,
  'center_y': 0.373567,
  'height': 0.024851,
  'width': 0.090401}}

In [10]:
data = dataset

#### 4. Creating a function for storing border coordinates of license plates

In [11]:
def details(x,flag_2):
        co=x['relative_coordinates']
        wid=float(co['width'])/2.0
        hi=float(co['height'])/2.0
        if flag_2=='y2':
            return float(co['center_y'])+hi
        if flag_2=='x2':
            return float(co['center_x'])+wid
        if flag_2=='x1':
            return float(co['center_x'])-wid
        if flag_2=='y1':
            return float(co['center_y'])-hi

data['x1'] = data['plate_data'].transform(lambda f:details(f,'x1'))    
data['y1'] = data['plate_data'].transform(lambda f:details(f,'y1'))  
data['x2'] = data['plate_data'].transform(lambda f:details(f,'x2'))  
data['y2'] = data['plate_data'].transform(lambda f:details(f,'y2'))

#### 5. Cropping as per the border coordinates area

In [12]:
from PIL import Image

In [13]:
count=0
for x,y in data.iterrows():
        img = Image.open('/content/drive/My Drive/Plates_Crop/img2/'+str(y['filename']))
        dim = img.size
        w,h = dim[0],dim[1]
        x1 = y['x1']*w
        x2 = y['x2']*w
        y1 = y['y1']*h
        y2 = y['y2']*h
        area = (x1,y1,x2,y2)
        print(count)
        cropped_img = img.crop(area)
        cropped_img = cropped_img.convert('RGB')
        cropped_img.save("/content/drive/My Drive/Plates_Crop/Cropped_Img/"+str(x)+'.jpg')
        count+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
